In [1]:
import json
import requests
import os
import pandas as pd
import time


from dotenv import load_dotenv
load_dotenv()


True

In [20]:
# secret api to get data from https://www.op.nysed.gov/

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    'Origin': 'https://www.op.nysed.gov',
    'Referer': 'https://www.op.nysed.gov/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'dnt': '1',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-gpc': '1',
    'x-oapi-key': '24tE+E/0NOuVZCsjLbMt3n5/2uJTYWqXhp/TiyL2A0z+I4ciWFJBMBIChlKGTBFU',
}

params = {
    'name': 'a',
    'pageNumber': '0',
    'pageSize': '200',
}

In [21]:
# make a list of all the letters in the alphabet
letters = [chr(i) for i in range(97, 123)]

letters

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [22]:
# loop through the list of letters and make a request for each letter

pharmacies = []

for i in letters:
    params['name'] = i

    response = requests.get(
        'https://api.nysed.gov/rosa/V2/findPharmacies', params=params, headers=headers)

    data = response.json()

    # obtain the data we want from the response
    # only run this for loop when data['last'] != False
    if data['last'] != True:

        for i in data['content']:

            registration_number = i['registrationNumber']
            legal_name = i['legalName']['value']
            trade_name = i['tradeName']['value']
            address = i['address']['value']
            county = i['county']['value']
            status = i['status']['value']
            status_label = i['status']['label']
            date_first_registered = i['dateFirstRegistered']['value']
            date_registered_begins = i['dateRegistrationBegins']['value']
            registered_through = i['dateRegisteredThrough']['value']
            deleted_date = i['deletedDate']

            # make a new dictionary with the data
            pharmacies_dict = {
                'registration_number': registration_number,
                'legal_name': legal_name,
                'trade_name': trade_name,
                'address': address,
                'county': county,
                'status': status,
                'status_label': status_label,
                'date_first_registered': date_first_registered,
                'date_registered_begins': date_registered_begins,
                'registered_through': registered_through,
                'deleted_date': deleted_date
            }

            print(pharmacies_dict)
            # append the new dictionary to the list of pharmacies
            pharmacies.append(pharmacies_dict)

pharmacies


[]

In [15]:
pharmacies

[{'registration_number': '038184',
  'legal_name': '1 PLUS PHARMACY, CORP.',
  'trade_name': None,
  'address': '29 EAST BROADWAY NEW YORK NY 10002',
  'county': 'NEW YORK',
  'status': 'Not Active',
  'status_label': 'Establishment Status',
  'date_first_registered': 'Not On File',
  'date_registered_begins': None,
  'registered_through': 'September 23, 2020',
  'deleted_date': 'September 23, 2020'},
 {'registration_number': '028701',
  'legal_name': '1 STOP PHARMACY & FOOD MART, INC.',
  'trade_name': None,
  'address': 'C/O PHARMACY 71-17 ROOSEVELT AVE JACKSON HEIGHTS NY 11372',
  'county': 'QUEENS',
  'status': 'Discontinued',
  'status_label': 'Establishment Status',
  'date_first_registered': 'December 12, 2007',
  'date_registered_begins': 'December 12, 2007',
  'registered_through': 'March 16, 2009',
  'deleted_date': 'March 16, 2009'},
 {'registration_number': '017003',
  'legal_name': '10 MONROE DRUG CORP.',
  'trade_name': 'VILLAGE SHOPPE PHARMACY',
  'address': '61 CATHERIN

In [16]:
# convert the list of pharmacies to a dataframe
import pandas as pd

pharmacies_df = pd.DataFrame(pharmacies)

# save the dataframe to a csv file

pharmacies_df.to_csv('ny_pharmacies_unedited.csv', index=False)

In [17]:
pharmacies_df = pd.read_csv('ny_pharmacies_unedited.csv')

In [18]:
# remove the pharmacies that are not active

pharmacies_df = pharmacies_df[pharmacies_df['status'] == 'Active']


In [19]:
# remove duplicate rows in the dataframe

pharmacies_df = pharmacies_df.drop_duplicates()

pharmacies_df

,registration_number,legal_name,trade_name,address,county,status,status_label,date_first_registered,date_registered_begins,registered_through,deleted_date
3,19404,"10 PARK PHARMACY, INC.",PASTEUR PHARMACY,53 E. 34TH ST. NEW YORK NY 10016,NEW YORK,Active,Establishment Status,"March 23, 1987","April 01, 2022","March 31, 2025",NaN
4,39919,100 CARE PHARMACY INC.,NaN,36-42 UNION STREET FLUSHING NY 11354,QUEENS,Active,Establishment Status,"November 09, 2022","November 09, 2022","October 31, 2025",NaN
7,38525,101 RX PHARMACY INC,NaN,9718 101ST AVE. OZONE PARK NY 11416,QUEENS,Active,Establishment Status,"February 19, 2021","February 19, 2021","January 31, 2024",NaN
14,33179,1070 SOUTHERN DRUG LLC,BOCA PHARMACY,1070 SOUTHERN BLVD BRONX NY 10459-2499,BRONX,Active,Establishment Status,"December 01, 2014","December 01, 2020","November 30, 2023",NaN
15,39246,108 PHARMACY INC.,NaN,45-02 108TH STREET NEW YORK NY 11368,NEW YORK,Active,Establishment Status,"December 15, 2021","December 15, 2021","November 30, 2024",NaN
...,...,...,...,...,...,...,...,...,...,...,...
5183,30243,GENZYME CORPORATION,NaN,450 WATER STREET CAMBRIDGE MA 02141,NaN,Active,Establishment Status,"July 02, 2010","July 01, 2022","June 30, 2025",NaN
5193,33940,GOODMAN PLAZA PHARMACY INC.,NaN,3250 THIRD AVENUE BRONX NY 10456,BRONX,Active,Establishment Status,"November 24, 2015","November 01, 2021","October 31, 2024",NaN
5195,39442,GRUM CORPORATION,AZE PHARMACY,4519 7TH AVE BROOKLYN NY 11220,KINGS,Active,Establishment Status,"April 27, 2022","April 27, 2022","March 31, 2025",NaN
5196,29983,GSR PHARMACY INC.,EZ PHARMACY & SURGICALS,8-03 ASTORIA BLVD. ASTORIA NY 11102,QUEENS,Active,Establishment Status,"March 24, 2010","March 01, 2022","February 28, 2025",NaN


In [8]:
addresses = pharmacies_df["address"].tolist()

In [9]:
api_key = os.getenv('key')

In [10]:
# Use Google Maps API to get the latitude and longitude of each address

addresses_info = []

#looping through the placeNames list
for place in addresses:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    lat = result_dic['results'][0]['geometry']['location']['lat']
    long = result_dic['results'][0]['geometry']['location']['lng']
    info  = {'address': place, 'lat': lat, 'long': long}
    addresses_info.append(info)
    print(info)
    time.sleep(0.25)

{'address': '53 E. 34TH ST. NEW YORK NY 10016', 'lat': 40.7476342, 'long': -73.981923}
{'address': '36-42 UNION STREET FLUSHING NY 11354', 'lat': 40.7626309, 'long': -73.8278344}
{'address': '9718 101ST AVE. OZONE PARK NY 11416', 'lat': 40.6847878, 'long': -73.8433708}
{'address': '1070 SOUTHERN BLVD BRONX NY 10459-2499', 'lat': 40.8253367, 'long': -73.8915682}
{'address': '45-02 108TH STREET NEW YORK NY 11368', 'lat': 40.747696, 'long': -73.85754899999999}
{'address': '1105 JERICHO TPKE NEW HYDE PARK NY 11040', 'lat': 40.7328929, 'long': -73.6847765}
{'address': '1150 WEBSTER AVE. BRONX NY 10456-5205', 'lat': 40.8302967, 'long': -73.9103745}
{'address': '3348 12 MILE ROAD BERKLEY MI 48072', 'lat': 42.5031325, 'long': -83.1930959}
{'address': '764 BURKE AVENUE BRONX NY 10467', 'lat': 40.8712539, 'long': -73.8645248}
{'address': '37-12 PRINCE ST. RETAIL F14 FLUSHING NY 11354', 'lat': 40.7604178, 'long': -73.8331616}
{'address': '3206 MERMAID AVE. BROOKLYN NY 11224-1810', 'lat': 40.57497

In [11]:
# check that the number of addresses matches the number of pharmacies

len(addresses_info)


464

In [12]:
# append the latitude and longitude to the dataframe

pharmacies_df['latitude'] = [i['lat'] for i in addresses_info]
pharmacies_df['longitude'] = [i['long'] for i in addresses_info]


In [13]:
# save the dataframe to a csv file

pharmacies_df.to_csv('ny_pharmacies_clean.csv', index=False)